# Creating a random Tracking ID

Goal: a short (10 digit) random ID to use to identify and track our messges through the process.  

Notes:
* Implements Python __secrets__
* The generated tracking ID is a relatively small sequence of unique combination of 10 upper case ascii letters and the digits 0-9.  
* Randomness here is the binomial coefficent of:  
36 choose 8 $$ \binom{36}{8} $$   
resulting in a possible ___30,260,340___ combinations.

## Show your work...   
To begin, I wanted to find 8 random bytes using the secrets module in Python.

In [103]:
                                                                                                                                                                                                                                                                                                                                                                    import secrets
n=8
secrets.token_bytes(n)


b'\xcd\xb7B\xeb\x85\xc8d\xcb'

Those characters are represented in binary, or as Python refers to them "byte-like objects," which isn't the most user friendly.

Let's convert it to a two character hexadecimal number.

In [ ]:
import secrets
n=8
secrets.token_bytes(n).hex()


: 

That's better, but there's an easier way to do it.... 

In [105]:
import secrets
n=8
secrets.token_hex(n)



'b9eed7d9bec8c216'

Show me what eight of varying lengths would look like.

In [3]:

import secrets
n = 8
for i in range (8):
    print(str(i) + " : " + secrets.token_bytes(i).hex())

0 : 
1 : 78
2 : a5b9
3 : 12708e
4 : b4c4ef3e
5 : b0c57e86de
6 : c78fce834517
7 : eb4dd2a3efc848


Hex is still too unfriendly for what I want to accmomplish.  Let's set up a domain of possible values being the captial letters and the digits 0-9.

In [5]:
import string
alphabet = string.ascii_letters + string.digits
print(alphabet)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789


How about just uppercase letters and digits 0-9

In [17]:
import string
alphabet = string.ascii_uppercase + string.digits
print(alphabet)

ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789


Good.  Now concatenate a string of them together.

In [20]:
import string
alphabet = string.ascii_uppercase + string.digits
tracking_number = ''
for i in range (8):
    tracking_number = tracking_number + (secrets.choice(alphabet))
print (tracking_number)

DIWQJQ69


Looks good, but make the string a little longer and show me a list of 100 so I can eyeball that they really do appear random.   
And we're done.

In [23]:
import string
alphabet = string.ascii_uppercase + string.digits
for i in range (25):
    tracking_number = ''
    for i in range (10):
        tracking_number = tracking_number + (secrets.choice(alphabet))
    print (tracking_number)

7KT54YNVEK
HEZHUM7ML8
TAS5WYFQDT
K1G30Y1XL4
OXTWCF6LEC
ID0KNILBJS
0GR665QKFD
KEFZJ0VSG2
YDN03595K1
KVM2EXUG3A
ON6SQDSZ21
O12EIRGURE
4LDTGP1G3X
I7AF6QR9ZK
THXWQV8L0Z
IYCD9ZCLY5
RQ7Q4YP8K3
7KJ4O498Z2
7SALOP2ODW
XIDCR3LTQL
9RWRZSLLYW
XMQJPX53XC
NGN37EM7BK
E64S4THJQJ
1XIKSFJ6UM


In [10]:
#!/usr/bin/env python3
#
# File: app_logging.py
# About: Logging provider
# Development: Kelly Kinney, Leo Rubiano
# Date: 2021-07-16 (July 16th, 2021)
#
"""
A class to standardize log formatting across all application artifacts.

Define common loggers and format to be used across the application.
NOTE: These logs are localized and non-persistent.
If used with a Docker container,
they cease to exist when the container does.

    Usage: (not meant to be called directly)
    log = create_logger("app_logging")
    log.debug("This message will be logged.")

"""
import logging
import os
from os.path import join, dirname
from dotenv import load_dotenv


def create_logger(name):
    """This function creates a logger template for the discovery package.

    This function creates a consistant format and location for
    all application log files to write to.
    """
    dotenv_path = join(dirname(__file__), '../.env')
    load_dotenv(dotenv_path)
    discovery_log_file = str(os.getenv("discovery_LOG_FILE"))
    web_response_file = str(os.getenv("WEB_RESPONSE_FILE"))

    logger = logging.getLogger(name)

    # create file handler which writes to an application log view.
    file_logger = logging.FileHandler(discovery_log_file)
    file_logger.setLevel(logging.INFO)

    # create console handler which writes to the console
    console_logger = logging.StreamHandler()
    console_logger.setLevel(logging.INFO)

    # create a logger to handle responses
    response_logger = logging.FileHandler(web_response_file)
    response_logger.setLevel(logging.INFO)

    # Create a custom formatter and add it to the handlers
    _format = "%(asctime)s - %(levelname)s - %(name)s - %(message)s"
    datefmt = "%m/%d/%Y %I:%M:%S %p"
    formatter = logging.Formatter(_format, datefmt)

    file_logger.setFormatter(formatter)
    console_logger.setFormatter(formatter)

    # Create a custom formatter for the response_logger
    response_format = "%(asctime)s - %(message)s"
    response_datefmt = "%Y%m%d:%H:%M:%S"
    response_formatter = logging.Formatter(response_format,
                                           response_datefmt)

    response_logger.setFormatter(response_formatter)

    # Associate the the handlers to the loggers
    logger.addHandler(file_logger)
    logger.addHandler(console_logger)
    logger.addHandler(response_logger)

    return logger


In [13]:
#!/usr/bin/env python3
# pylint: disable=R0903
# Too few public methods.
# File: create_urn_id.py
# About: Generate a unique identifier for the discovery to use to
# track through the process.
# Development: Kelly Kinney, Leo Rubiano
# Date: 2021-11-05 (November 11th, 2021)
"""This class creates a random 10 character alphanumeric id to
track the request
through the process end-to-end."""
import secrets
import string
# from app_logging import create_logger


class CreateTrackingID:
    """Class to create a tracking Id for the discovery."""

    def __init__(self):
        """Entry point into the application and define variables upon
        class instantiation."""
        self.log = create_logger("create_tracking_id")
        self.tracking_id = ""
        self.alphabet = string.ascii_uppercase + string.digits
        self.log.info(
            f"Created domain of allowed characters for tracking id: {self.alphabet}."
        )

    def create_tracking_id(self, id_size):
        """Creates a completely random uuid not associated with mac address,
        node or timestamp."""
        for _ in range(id_size):
            self.tracking_id = self.tracking_id +\
                (secrets.choice(self.alphabet))
        return self.tracking_id
